In [ ]:
import json

with open("essential_account.json","r",encoding="UTF-8") as file:
    data = json.load(file)
    accounts = data["essential_accounts"]

for account in accounts:
    print(account)
    account["name"]




In [1]:
import pandas as pd 

from player import *

game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager
product_A = game_master.construct_instance("inventory","product_A").get("ID")
asset_list = list(game_master.asset_registry.keys())
player1.redister_product(product_A)

player1.purchase_product(product_A, 100, 150)
player1.purchase_product(product_A, 200, 120, 15)

player1.sale_product(product_A, 80)

player1.purchase_product(product_A, 30, 160)
player1.sale_product(product_A, 100, 225)

player1.sale_product(product_A, 100, 210, 500)

player1.perform_inventory_audit(product_A)

end_1 = ledger1.execute_settlement()

ledger1.display_transaction_history()
# ledger1.display_trial_balance(end_1)
# ledger1.display_financial_statements(end_1)

df = pd.DataFrame.from_dict(end_1, orient="index")


資産 'product_A' (ID: 3461bcb4-3bd3-45cd-87d7-e593144153f1, クラス: <class 'asset.Inventory'>) が登録されました。
[player1]**商品が登録されました** 商品名：product_A
[player1,product_A]**売価が更新されました**　更新後：225
[player1,product_A]**売価が更新されました**　更新後：210


取引一覧:

  [ゲーム内時間]
    仕訳: 現金: 5000, 資本金: -5000
    摘要: Initial capital
  [ゲーム内時間]
    仕訳: 仕入: 15000, 現金: -15000
    摘要: 商品の仕入れ　商品名：product_A 個数：100 単価：150
  [ゲーム内時間]
    仕訳: 仕入: 24015, 現金: -24015
    摘要: 商品の仕入れ　商品名：product_A 個数：200 単価：120
  [ゲーム内時間]
    仕訳: 現金: 0, 売上高: 0
    摘要: 商品の売上 商品名：product_A 個数：80 単価：0
  [ゲーム内時間]
    仕訳: 仕入: 4800, 現金: -4800
    摘要: 商品の仕入れ　商品名：product_A 個数：30 単価：160
  [ゲーム内時間]
    仕訳: 現金: 22500, 売上高: -22500
    摘要: 商品の売上 商品名：product_A 個数：100 単価：225
  [ゲーム内時間]
    仕訳: 現金: 20500, 売上高: -20500
    摘要: 商品の売上 商品名：product_A 個数：100 単価：210
  [ゲーム内時間]
    仕訳: 売上原価: 34565, 仕入: -43815, 棚卸減耗: 0, 商品評価損: 0, 棚卸資産: 9250
    摘要: 棚卸調整 商品: product_A


In [2]:
ledger1.display_financial_statements(end_1)


=== 貸借対照表 ===
資産:
  流動資産:
        現金: 4185
        棚卸資産: 9250
  固定資産:
  繰延資産:
負債:
  流動負債:
  固定負債:
純資産:
  株主資本:
        資本金: (5000)
  評価・換算差額:

=== 損益計算書 ===
収益:
  営業収益:
        売上高: (43000)
  営業外収益:
費用:
  営業費用:
        売上原価: 34565
  営業外費用:

当期純利益: 8,435


In [4]:
inventory_c = game_master.construct_instance("inventory", "inventory_c")
type(inventory_c.get("class"))


資産 'inventory_c' (ID: f06b0cdf-4b83-4f4c-b3ae-4b372e7f2641, クラス: <class 'asset.Inventory'>) が登録されました。


type

In [ ]:
import pandas as pd 

from player import *

game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager

building_B = game_master.construct_instance("building")

In [ ]:
from asset import *

building_A = Building("building_A", 5000, "player1", "東京都新宿区")
building_A.apply_depreciation()